In [43]:
import numpy as np
import astropy.io.fits as fits
from astropy.table import Table
from astromodels import Powerlaw,  PointSource, SpectralComponent
import astropy.time as at
from datetime import datetime
from pyspi.utils.function_utils import find_response_version
from pyspi.utils.response.spi_response_data import ResponseDataRMF
from pyspi.utils.response.spi_response import ResponseRMFGenerator
from pyspi.utils.response.spi_drm import SPIDRM
from pyspi.utils.livedets import get_live_dets
import os
from MultinestClusterFit import powerlaw_binned_spectrum

In [44]:
# put original data files from afs in this folder
# rename evts_det_spec.fits to evts_det_spec_orig.fits

# data_path = "crab_data/0374"
data_path = "crab_data/1380"

In [45]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])

In [46]:
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])

In [47]:
# Only necessary for 1380
skip_pointing = [False] * len(pointings)
skip_pointing[0] = True

In [48]:
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])

In [49]:
# Define Source and Spectrum

# ra, dec = 10, -40
# K, piv, index = 6e-3, 40, -2
ra, dec = 155., 75.
K, piv, index = 3e-3, 40, -1
    
pl = Powerlaw()
pl.piv = piv
pl.K = K
pl.index = index
component1 = SpectralComponent("pl", shape=pl)
source = PointSource("Test", ra=ra, dec=dec, components=[component1])

emod = np.geomspace(10, 3000, 50)
spec = source(emod)
spec_binned = powerlaw_binned_spectrum(emod, spec)
# spec_binned = (emod[1:]-emod[:-1])*(spec[:-1]+spec[1:])/2

In [8]:
# Generate Source Counts

assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)

source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], ra, dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mat = sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1))
    
    count_rates = np.dot(spec_binned, resp_mat)
    
    for d_i, d in enumerate(dets):
        index = p_i * 85 + d
        source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[index])    


Using the irfs that are valid between 04/07/17 08:20:06 and 09/02/19 09:59:57 (YY/MM/DD HH:MM:SS)


In [10]:
# Save Data for PySpi

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
    counts = t
    
updated_counts = counts.copy()
updated_counts["COUNTS"] += source_counts

hdu = fits.BinTableHDU(data=updated_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{data_path}/evts_det_spec.fits")


In [ ]:
# Sim source data for spimodfit

In [11]:
# put spimodfit spi/ files in this folder
# rename evts_det_spec.fits.gz to evts_det_spec_old.fits.gz

# spimodfit_folder = "crab_data/0374_spimodfit"
spimodfit_folder = "crab_data/1380_spimodfit"

if not os.path.exists(spimodfit_folder):
    os.mkdir(spimodfit_folder)

In [12]:
with fits.open(f"{spimodfit_folder}/pointing.fits.gz") as file:
    t = Table.read(file[1])
    
spimodfit_pointings = np.array(t["PTID_ISOC"])

with fits.open(f"{spimodfit_folder}/energy_boundaries.fits.gz") as file:
    t = Table.read(file[1])
    
spimodfit_energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])


In [13]:
# Pointing Indices
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    p_indices = []
    p_rel_indices = []
    for i, p in enumerate(spimodfit_pointings):
        temp = np.argwhere(t["PTID_ISOC"]==p)
        if len(temp)>0:
            p_indices.append(temp[0][0])
            p_rel_indices.append(i)

# Energy Indices
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    e_indices = []
    for e in spimodfit_energy_bins:
        temp = np.argwhere(t["E_MIN"]==e)
        if len(temp)>0:
            e_indices.append(temp[0][0])
            
# Single Event Indices
se_indices = []
for i in p_indices:
    for j in range(19):
        se_indices.append(i*85 + j)
        
# Relative Pointing Deterctor Indices
rd_indices = []
for i in p_rel_indices:
    for j in range(19):
        rd_indices.append(i*19 + j)

In [14]:
# evts_det_spec.fits.gz
with fits.open(f"{data_path}/evts_det_spec.fits") as file:
    t = Table.read(file[1])
    
with fits.open(f"{spimodfit_folder}/evts_det_spec_old.fits.gz") as file:
    d = Table.read(file[1])
    h = file[1].header
    
eds_temp = t[se_indices]


for i in range(len(e_indices)-1):
    d["COUNTS"][:,i] = np.sum(eds_temp["COUNTS"][ : , e_indices[i] : e_indices[i+1]], axis=1)
    
hdu = fits.BinTableHDU(data=d, name="SPI.-OBS.-DSP")
hdu.header = h
hdu.writeto(f"{spimodfit_folder}/evts_det_spec.fits.gz")



In [ ]:
# replace evts_det_spec.fits.gz in spimodfit to run fit

In [50]:
# Spimodfit backgroud for PySpi
# put bg_e0020-0600/ in this folder
# Manually copy energy_boundaries.fits from spimodfit data

data_path2 = "crab_data/1380_spimodfit_bkg"

if not os.path.exists(data_path2):
    os.mkdir(data_path2)

In [17]:
with fits.open(f"{data_path2}/bg-e0020-0600/output_bgmodel-conti.fits.gz") as file:
    t = Table.read(file[1])
    conti = t["COUNTS"]
    
with fits.open(f"{data_path2}/bg-e0020-0600/output_bgmodel-lines.fits.gz") as file:
    t = Table.read(file[1])
    lines = t["COUNTS"]


In [18]:
new_energy_bins = spimodfit_energy_bins

In [19]:
# with fits.open(f"{data_path2}/energy_boundaries.fits") as file:
#     t = Table.read(file[1])
#     new_energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
# Energy Indices
energy_indices = []
for e in new_energy_bins:
    temp = np.argwhere(energy_bins==e)
    if len(temp)>0:
        energy_indices.append(temp[0][0])

# Pointing Indices
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    pointing_indices = []
    pointing_indices_full = []
    for i, p in enumerate(spimodfit_pointings):
        temp = np.argwhere(t["PTID_ISOC"]==p)
        if len(temp)>0:
            pointing_indices.append(temp[0][0])
            for j in range(85):
                pointing_indices_full.append(85*temp[0][0] + j)
            

        



In [20]:
new_source_counts = np.zeros((len(source_counts), len(new_energy_bins)-1))

for i in range(len(energy_indices)-1):
    new_source_counts[:,i] = np.sum(source_counts[ : , energy_indices[i] : energy_indices[i+1]], axis=1)
    
new_source_counts = new_source_counts[pointing_indices_full]

total_counts = new_source_counts.copy()

for i in range(len(pointing_indices)):
    total_counts[85*i : 85*i + 19, :] += (np.random.poisson(np.abs(lines[i*19: (i+1)*19])) * np.sign(lines[i*19: (i+1)*19])
                                          + np.random.poisson(np.abs(conti[i*19: (i+1)*19])) * np.sign(conti[i*19: (i+1)*19]))

In [21]:
with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
    counts = t
    
updated_counts = counts[pointing_indices_full]
updated_counts["COUNTS"] = total_counts

hdu = fits.BinTableHDU(data=updated_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{data_path2}/evts_det_spec.fits")

In [22]:
# dead_time.fits
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    
dt = t[pointing_indices_full]

hdu = fits.BinTableHDU(data=dt, name="SPI.-OBS.-DTI")
hdu.writeto(f"{data_path2}/dead_time.fits")

In [23]:
# pointing.fits
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
ps = t[pointing_indices]

hdu = fits.BinTableHDU(data=ps, name="SPI.-OBS.-PNT")
hdu.writeto(f"{data_path2}/pointing.fits")

In [24]:
# spimodfit with spimodfit bkg

if not os.path.exists(f"{spimodfit_folder}/w_smf_bkg"):
    os.mkdir(f"{spimodfit_folder}/w_smf_bkg")

with fits.open(f"{data_path2}/evts_det_spec.fits") as file:
    t = Table.read(file[1])
    
with fits.open(f"{spimodfit_folder}/evts_det_spec_old.fits.gz") as file:
    d = Table.read(file[1])
    h = file[1].header

smf_bkg_indices = []
for i in range(len(pointing_indices)):
    for j in range(19):
        smf_bkg_indices.append(i*85 + j)

d["COUNTS"] = np.array(t["COUNTS"][smf_bkg_indices], dtype=np.uint32)

hdu = fits.BinTableHDU(data=d, name="SPI.-OBS.-DSP")
hdu.header = h
hdu.writeto(f"{spimodfit_folder}/w_smf_bkg/evts_det_spec.fits.gz")

In [77]:
# PySpi with constant background
# Copy energy_boundaries, pointings into this folder

# pointing_index = 1 # 0374
pointing_index = 4 # 1380

# data_path3 = "crab_data/0374_const_bkg"
data_path3 = "crab_data/1380_const_bkg"

if not os.path.exists(f"{data_path3}"):
    os.mkdir(f"{data_path3}")

In [78]:
# Energy Bins
with fits.open(f"{data_path}/energy_boundaries.fits") as file:
    t = Table.read(file[1])
    energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])

In [79]:
# Pointings and Start Times
with fits.open(f"{data_path}/pointing.fits") as file:
    t = Table.read(file[1])
    
    pointings = np.array(t["PTID_SPI"])
    
    time_start = np.array(t["TSTART"]) + 2451544.5
    time_start = [at.Time(f"{i}", format="jd").datetime for i in time_start]
    time_start = np.array([datetime.strftime(i,'%y%m%d %H%M%S') for i in time_start])

In [80]:
# Time Elapsed
# det=i, pointing_index=j : index = j*85 + i
with fits.open(f"{data_path}/dead_time.fits") as file:
    t = Table.read(file[1])
    time_elapsed = np.array(t["LIVETIME"])
    
updated_time = t.copy()
    
for i in range(int(len(time_elapsed) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_time[i*85 : (i+1)*85] = updated_time[pointing_index*85 : (pointing_index+1)*85]
        
hdu = fits.BinTableHDU(data=updated_time, name="SPI.-OBS.-DTI") # is all of this correct?
hdu.writeto(f"{data_path3}/dead_time.fits")

In [81]:
# Generate Source Counts

assert find_response_version(time_start[0]) == find_response_version(time_start[-1]), "Versions not constant"
version = find_response_version(time_start[0])
rsp_base = ResponseDataRMF.from_version(version)

source_counts = np.zeros((len(pointings)*85, len(energy_bins)-1), dtype=np.uint32)

for p_i, pointing in enumerate(pointings):
    if skip_pointing[p_i]:
        continue
    
    time = time_start[p_i]
    dets = get_live_dets(time=time, event_types=["single"])
    
    rmfs = []
    for d in dets:
        rmfs.append(ResponseRMFGenerator.from_time(time, d, energy_bins, emod, rsp_base))
        
    sds = np.empty(0)
    for d in range(len(dets)):
        sd = SPIDRM(rmfs[d], ra, dec)
        sds = np.append(sds, sd.matrix.T)
    resp_mat = sds.reshape((len(dets), len(emod)-1, len(energy_bins)-1))
    
    count_rates = np.dot(spec_binned, resp_mat)
    
    for d_i, d in enumerate(dets):
        index = p_i * 85 + d
        source_counts[index,:] = np.random.poisson(count_rates[d_i,:] * time_elapsed[pointing_index*85 + d])    

Using the irfs that are valid between 10/05/27 12:45:00 and present (YY/MM/DD HH:MM:SS)


In [73]:
# Save Data for PySpi

with fits.open(f"{data_path}/evts_det_spec_orig.fits") as file:
    t = Table.read(file[1])
    
    counts = t
    
updated_counts = counts.copy()

for i in range(int(len(updated_counts) / 85)):
    if i == pointing_index:
        continue
    else:
        updated_counts[i*85 : (i+1)*85] = updated_counts[pointing_index*85 : (pointing_index+1)*85]
        
updated_counts["COUNTS"] = np.random.poisson(updated_counts["COUNTS"])

updated_counts["COUNTS"] += source_counts

hdu = fits.BinTableHDU(data=updated_counts, name="SPI.-OBS.-DSP")
hdu.writeto(f"{data_path3}/evts_det_spec.fits")

In [22]:
updated_counts

COUNTS,STAT_ERR
uint32[3964],float32[3964]
128 .. 1,8.774964 .. 1.0
77 .. 1,8.426149 .. 1.0
0 .. 0,0.0 .. 0.0
191 .. 1,12.288206 .. 1.0
108 .. 2,8.485281 .. 1.4142135
159 .. 1,11.269427 .. 1.0
100 .. 2,7.3484693 .. 1.4142135
76 .. 2,8.5440035 .. 1.4142135
79 .. 3,8.185352 .. 1.7320508


In [75]:
time_elapsed[4*85]

2514.8085787542122